In [1]:
!pip install nltk
!pip install tensorflow


In [2]:
import json
import random
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD


In [3]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [4]:
import nltk
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import nltk
nltk.download('punkt', download_dir='/usr/local/share/nltk_data')


[nltk_data] Downloading package punkt to /usr/local/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
import nltk
try:
    nltk.data.find('tokenizers/punkt')
    print("Punkt resource found!")
except LookupError:
    print("Punkt resource not found.")


Punkt resource found!


In [8]:

  >>> import nltk
  >>> nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [9]:
from nltk.tokenize import word_tokenize

# Test tokenization
test = word_tokenize("This is a test sentence.")
print(test)


['This', 'is', 'a', 'test', 'sentence', '.']


In [10]:
import nltk
nltk.download('wordnet', download_dir='/usr/local/share/nltk_data')


[nltk_data] Downloading package wordnet to
[nltk_data]     /usr/local/share/nltk_data...


True

In [11]:
data_file = open('intents.json').read()
intents = json.loads(data_file)


In [12]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenizing each word in the sentence
        w = nltk.word_tokenize(pattern)
        words.extend(w)

        # Adding the sentence and its tag to the documents
        documents.append((w, intent['tag']))

        # Add the tag to classes if it's not already in it
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatizing and removing duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(words), "unique lemmatized words")
print(len(classes), "classes", classes)


161 documents
107 unique lemmatized words
51 classes ['Adventure stories', 'American fiction', 'Architecture', 'Art', 'Biography & Autobiography', 'Body, Mind & Spirit', 'Business & Economics', "Children's stories", 'Comics & Graphic Novels', 'Computers', 'Cooking', 'Detective and mystery stories', 'Drama', 'Education', 'English fiction', 'Family & Relationships', 'Fantasy fiction', 'Fiction', 'Foreign Language Study', 'Games', 'Health & Fitness', 'History', 'Humor', 'Juvenile Fiction', 'Juvenile Nonfiction', 'Language Arts & Disciplines', 'Law', 'Literary Collections', 'Literary Criticism', 'Medical', 'Music', 'Nature', 'Performing Arts', 'Philosophy', 'Photography', 'Poetry', 'Political Science', 'Psychology', 'Religion', 'Science', 'Science fiction', 'Self-Help', 'Social Science', 'Sports & Recreation', 'Travel', 'True Crime', 'Young Adult Fiction', 'book_search', 'goodbye', 'greeting', 'thanks']


In [13]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [14]:
import random

training = []
output_empty = [0] * len(classes)

# Assuming 'documents' is your dataset with [pattern_words, category]
for doc in documents:
    # initializing bag of words
    bag = []

    # list of tokenized words for the pattern
    pattern_words = doc[0]

    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        if w in pattern_words:
            bag.append(1)
        else:
            bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features
random.shuffle(training)

# Create train and test lists. X - patterns, Y - intents
train_x = [item[0] for item in training]
train_y = [item[1] for item in training]

print(f"Training data length: {len(training)}")
print(f"Train X shape (patterns): {len(train_x)}, Train Y shape (intents): {len(train_y)}")


Training data length: 161
Train X shape (patterns): 161, Train Y shape (intents): 161


In [15]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons, and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Print model summary
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        13,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 51)             │         3,315 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,395 (99.20 KB)

 Trainable params: 25,395 (99.20 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.0398 - loss: 3.9644
Epoch 2/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0404 - loss: 3.9189
Epoch 3/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0740 - loss: 3.8506
Epoch 4/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0729 - loss: 3.8271
Epoch 5/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0949 - loss: 3.7772
Epoch 6/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1064 - loss: 3.6908
Epoch 7/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1026 - loss: 3.6343
Epoch 8/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1298 - loss: 3.4940
Epoch 9/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1165 - loss: 3.5344
Epoch 10/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1623 - loss: 3.2658
Epoch 11/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2098 - loss: 3.1835
Epoch 12/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy:

model created


In [17]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

    return sentence_words

In [18]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:

                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [19]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    return return_list

In [20]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break

    return result

In [21]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [25]:
chatbot_response('Recommend a book in history')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


{'Book': 'The Chaneysville Incident',
 'Feedback': "The legends say something happened in Chaneysville. The Chaneysville Incident is the powerful story of one man's obsession with discovering what that something was--a quest that takes the brilliant and bitter young black historian John Washington back through the secrets and buried evil of his heritage. Returning home to care for and then bury his father's closest friend and his own guardian, Old Jack Crawley, he comes upon the scant records of his family's proud and tragic history, which he drives himself to reconstruct and accept. This is the story of John's relationship with his family, the town, and the woman he loves; and also between the past and the present, between oppression and guilt, hate and violence, love and acceptance.",
 'Rate': 3.96}

In [26]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00


In [40]:
import gradio as gr
import nltk
import numpy as np
import random
import json
from nltk.stem import WordNetLemmatizer
from keras.models import load_model

# Load the trained chatbot model and data
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())

lemmatizer = WordNetLemmatizer()

# General book suggestions for general queries
general_book_suggestions = [
    {
        "Book": "To Kill a Mockingbird",
        "Feedback": "A powerful novel that explores the racial injustice in the American South.",
        "Rate": 4.28
    },
    {
        "Book": "1984",
        "Feedback": "A dystopian novel about a totalitarian regime that uses surveillance to control society.",
        "Rate": 4.19
    },
    {
        "Book": "Pride and Prejudice",
        "Feedback": "A classic novel of manners that explores themes of love, class, and society.",
        "Rate": 4.25
    }
]

# Function to process the input message and predict the response
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']

    # Handle greeting queries
    if tag == "greeting":
        return random.choice(["Hello! How can I help you today?", "Hi there! What can I do for you?", "Hey! What brings you here today?"])

    # Handle goodbye queries
    elif tag == "goodbye":
        return random.choice(["Goodbye! If you have any more questions, feel free to ask.", "Bye! Have a great day!"])

    # Handle thanks queries
    elif tag == "thanks":
        return random.choice(["You're welcome! If you need assistance, feel free to ask.", "No problem! Let me know if there's anything else I can help with."])

    # Handle book search queries
    elif tag == "book_search":
        return "Certainly! What genre are you interested in? Or if you want a random book recommendation, I can help with that as well!"

    # Handle general book suggestions (if no genre is specified)
    if tag == "general":
        return random.choice(general_book_suggestions)

    # Otherwise, process the genre-specific response
    for i in list_of_intents:
        if i['tag'] == tag:
            # If responses are a list of books, select one book randomly
            if isinstance(i['responses'], list):
                books = random.choice(i['responses'])  # This is a list of books
                result = random.choice(books)  # Randomly pick one book from this list
            else:
                result = i['responses']  # In case only one book is listed for that tag
            break

    # If no books are found in the genre (fallback), suggest a related genre or random book
    if isinstance(result, list):
        # In case there are no books for that genre, suggest a general book instead
        result = random.choice(general_book_suggestions)

    return result

# Function that connects everything
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)

    if isinstance(res, dict):
        return f"📖 Book: {res.get('Book')}\n📝 Feedback: {res.get('Feedback')}\n⭐ Rating: {res.get('Rate')}"
    return res

# Launch the Gradio interface with chatbot_response function
gr.Interface(fn=chatbot_response, inputs="text", outputs="text", title="AI Book Recommender", theme="soft").launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dd2f452427d6ba8fb0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
